In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))

In [5]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# data set oluşturalım
block_size = 3 
X, Y = [], []
for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '------>', itos[ix] )
        context = context[1:] + [ix] 
X = torch.tensor(X)
Y = torch.tensor(Y)

In [7]:
X.shape

torch.Size([228146, 3])

In [8]:
C = torch.randn((27,2))

In [9]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [10]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [11]:
h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1)

In [12]:
h.shape

torch.Size([228146, 100])

In [13]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [14]:
logits = h @ W2 + b2

In [15]:
logits.shape

torch.Size([228146, 27])

In [16]:
counts = logits.exp()

In [17]:
prob = counts / counts.sum(1, keepdims=True)

In [18]:
prob.shape

torch.Size([228146, 27])

In [19]:
#prob[torch.arange(32), Y]

In [20]:
#loss = -prob[torch.arange(32), Y].log().mean()
#loss

In [21]:
# buda aynı işleme denk gelmektedir
#F.cross_entropy(logits, Y)
#loss

In [22]:
# şimdi herşeyi düzenli bir şekilde yazalım

g = torch.Generator().manual_seed(123456)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True


In [23]:
for _ in range(10):
    emb = C[X]
    h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)

    for p in parameters:
        p.grad = None
    loss.backward()


    for p in parameters:
        p.data = -0.1 * p.grad
print(loss.item())

3.291750192642212
